The purpose of this excercise is to demonstrate my knowledge in the underlying theory of the ordinary least squares (OLS) method of linear regression by applying the methods to a dataset obtained from Kaggle

In [1]:
#Part 1
#The part of this procedure involves preparing the data, through cleaning and creating new variables

#drop rows with missing variables
#drop categoricals
#drop year
#drop price in inr
#drop torque 

#fuel = dummy (1,2,3,4) for ('Diesel', 'Petrol', 'LPG', 'CNG')
#seller_type = dummy (1,2,3) for ('Individual', 'Dealer', 'Trustmark Dealer')
#transmission = dummy (1,2) for ('automatic','manual') 
#owner = dummy (1,2,3,4,5) for ('First Owner', 'Second Owner', 'Third Owner','Fourth & Above Owner', 'Test Drive Car')
    #drop the test drive car

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('cardetails.csv')

In [4]:
#Drop Missing Values
df = df.dropna(axis=0)

#deleting name/torque
del df['name']
del df['torque']
del df['year']

#Fuel Dummy
d = {'Diesel': 1, 'Petrol': 2, 'CNG':3, 'LPG':4,}
df['fuel_type'] = df['fuel'].map(d)
del df['fuel']

#Drop LPG and CNG
df = df[df.fuel_type != (3)]
df = df[df.fuel_type != (4)]

#Seller Dummy
d = {'Individual': 1, 'Dealer': 2, 'Trustmark Dealer': 3}
df['seller'] = df['seller_type'].map(d)
del df['seller_type']

#Transmission Dummy
d = {'Automatic': 1, 'Manual': 2}
df['transmission_type'] = df['transmission'].map(d)
del df['transmission']

#Dropping the 5 obs related to Test Drive Car
df = df[df.owner != 'Test Drive Car']

#Owner Dummy
d = {'First Owner': 1, 'Second Owner': 2, 'Third Owner': 3, 'Fourth & Above Owner': 4}
df['ownership'] = df['owner'].map(d)
del df['owner']

del df['selling_price_INR']

In [5]:
#convert string to float
    #mileage,engine,power
    #remove str elements from numerical variables

In [6]:
mileagedf = df.mileage.apply(lambda x: pd.Series(str(x).split(" ")))
del mileagedf[1]
mileagedf[0]=mileagedf[0].astype(float)
mileagedf.rename(columns={0: 'Kmpl'},inplace=True)

In [7]:
enginedf = df.engine.apply(lambda x: pd.Series(str(x).split(" ")))
del enginedf[1]
enginedf[0]=enginedf[0].astype(float)
enginedf.rename(columns={0: 'Engine(CC)'},inplace=True)

In [8]:
powerdf = df.max_power.apply(lambda x: pd.Series(str(x).split(" ")))
del powerdf[1]
powerdf[0]=powerdf[0].astype(float)
powerdf.rename(columns={0: 'Power(BHP)'},inplace=True)

In [9]:
df2=pd.concat([mileagedf,enginedf,powerdf],axis=1)

In [10]:
df = pd.concat([df,df2],axis=1)
del df['mileage']
del df['engine']
del df['max_power']

In [11]:
df.describe()

,age,selling_price_USD,km_driven,seats,fuel_type,seller,transmission_type,ownership,Kmpl,Engine(CC),Power(BHP)
count,7814.000000,7814.000000,7.814000e+03,7814.000000,7814.000000,7814.000000,7814.000000,7814.000000,7814.000000,7814.000000,7814.000000
mean,6.012926,8470.623565,6.916070e+04,5.421551,1.449962,1.201305,1.867289,1.443819,19.391962,1462.914640,91.890230
std,3.864426,10527.362419,5.687495e+04,0.963125,0.497522,0.470334,0.339283,0.704962,4.001972,504.759742,35.727695
min,0.000000,389.990000,1.000000e+03,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,624.000000,34.200000
25%,3.000000,3510.000000,3.400000e+04,5.000000,1.000000,1.000000,2.000000,1.000000,16.780000,1197.000000,69.000000
50%,5.000000,5915.000000,6.000000e+04,5.000000,1.000000,1.000000,2.000000,1.000000,19.300000,1248.000000,82.400000
75%,8.000000,9035.000000,9.600000e+04,5.000000,2.000000,1.000000,2.000000,2.000000,22.320000,1582.000000,102.000000
max,26.000000,130000.000000,2.360457e+06,14.000000,2.000000,3.000000,2.000000,4.000000,42.000000,3604.000000,400.000000


In [12]:
from scipy import stats
z = np.abs(stats.zscore(df))
print(z)

[[0.00334496 0.24895042 1.34231651 ... 1.00157983 0.42580336 0.50077047]
 [0.00334496 0.34774692 0.89393576 ... 0.43682212 0.06951348 0.32553218]
 [2.0669526  0.60955764 1.24560694 ... 0.42280907 0.06753221 0.38880534]
 ...
 [1.29059102 0.33292744 0.89393576 ... 0.02298061 0.42580336 0.5035696 ]
 [0.25544224 0.44654342 0.77650231 ... 1.0440616  0.13257579 0.6127356 ]
 [0.25544224 0.44654342 0.77650231 ... 1.0440616  0.13257579 0.6127356 ]]


In [13]:
threshold = 3
print(np.where(z > 3))

(array([   6,   16,   43, ..., 7764, 7801, 7810], dtype=int64), array([0, 0, 9, ..., 2, 0, 7], dtype=int64))


In [70]:
df2 = df[(z < 3).all(axis=1)]

In [71]:
df2.describe()

,age,selling_price_USD,km_driven,seats,fuel_type,seller,transmission_type,ownership,Kmpl,Engine(CC),Power(BHP)
count,6894.000000,6894.000000,6894.000000,6894.000000,6894.000000,6894.000000,6894.00000,6894.000000,6894.000000,6894.000000,6894.000000
mean,5.910647,6863.585049,67437.029301,5.373078,1.452567,1.132579,1.90499,1.401944,19.763396,1405.206701,87.899233
std,3.520040,5281.230273,42244.558197,0.850021,0.497781,0.339144,0.29325,0.612413,3.835007,443.461449,28.643558
min,0.000000,409.550000,1000.000000,4.000000,1.000000,1.000000,1.00000,1.000000,9.000000,624.000000,34.200000
25%,3.000000,3510.000000,35000.000000,5.000000,1.000000,1.000000,2.00000,1.000000,17.000000,1197.000000,68.050000
50%,5.000000,5850.000000,60000.000000,5.000000,1.000000,1.000000,2.00000,1.000000,19.700000,1248.000000,81.860000
75%,8.000000,8450.000000,94288.750000,5.000000,2.000000,1.000000,2.00000,2.000000,22.700000,1498.000000,100.000000
max,17.000000,39000.000000,239451.000000,8.000000,2.000000,2.000000,2.00000,3.000000,28.400000,2956.000000,198.250000


To ensure that 

In [73]:
df2.insert(0, 'Constant', 1)

In [75]:
df2

,Constant,age,selling_price_USD,km_driven,seats,fuel_type,seller,transmission_type,ownership,Kmpl,Engine(CC),Power(BHP)
0,1,6,5850.0,145500,5.0,1,1,2,1,23.40,1248.0,74.00
1,1,6,4810.0,120000,5.0,1,1,2,2,21.14,1498.0,103.52
2,1,14,2054.0,140000,5.0,2,1,2,3,17.70,1497.0,78.00
3,1,10,2925.0,127000,5.0,1,1,2,1,23.00,1396.0,90.00
4,1,13,1690.0,120000,5.0,2,1,2,1,16.10,1298.0,88.20
...,...,...,...,...,...,...,...,...,...,...,...,...
8122,1,6,6175.0,80000,5.0,1,1,2,2,22.54,1396.0,88.73
8123,1,7,4160.0,110000,5.0,2,1,2,1,18.50,1197.0,82.85
8125,1,11,4966.0,120000,5.0,1,1,2,1,19.30,1248.0,73.90
8126,1,7,3770.0,25000,5.0,1,1,2,1,23.57,1396.0,70.00


Now that we have cleaned the data we can proceed to applying the following Ordinary Least Squares formula and calculating the regression coefficients.

$$\hat{\beta}=(X'X)^{-1}X'Y$$

In [88]:
Y=df2['selling_price_USD']

In [89]:
X=df2[['Constant','age','km_driven','seats','fuel_type','seller','transmission_type','ownership','Kmpl','Engine(CC)','Power(BHP)']]

We now have matrices of $X$ and $Y$ of the following orders:
$$X=(6,894\times 11)$$
$$Y=(6,894\times 1)$$

In [90]:
Xt=X.transpose()

$$X'=(11 \times 6,894)$$

In [91]:
XtX=Xt.dot(X)

$$(X'X)=(11\times11)$$

Here we are computing the inverse of $(X'X)$.

In [93]:
XtX_inv = pd.DataFrame(np.linalg.pinv(XtX.values), XtX.columns, XtX.index)


To confirm we have the inverse, we take the product of the two matrices and recieve the identity matrix. $(X'X)(X'X)=I$


In [94]:
XtX.dot(XtX_inv).round()

,Constant,age,km_driven,seats,fuel_type,seller,transmission_type,ownership,Kmpl,Engine(CC),Power(BHP)
Constant,1.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
age,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0
km_driven,0.0,-0.0,1.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0
seats,0.0,-0.0,0.0,1.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0
fuel_type,0.0,-0.0,0.0,-0.0,1.0,0.0,0.0,0.0,-0.0,0.0,-0.0
seller,0.0,-0.0,0.0,-0.0,-0.0,1.0,0.0,0.0,-0.0,0.0,-0.0
transmission_type,0.0,-0.0,0.0,-0.0,-0.0,0.0,1.0,0.0,-0.0,-0.0,-0.0
ownership,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,1.0,-0.0,0.0,-0.0
Kmpl,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,1.0,0.0,-0.0
Engine(CC),0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,1.0,-0.0


Constructing the OLS estimator from the previous results: $\hat{\beta}=(X'X)^{-1}X'Y$


In [99]:
(XtX_inv).dot(Xt).dot(Y)

Constant             14476.816434
age                   -526.314936
km_driven               -0.018906
seats                 -127.268097
fuel_type            -2229.507294
seller                1103.130412
transmission_type    -2883.004021
ownership             -206.796781
Kmpl                  -124.823692
Engine(CC)               0.288325
Power(BHP)              82.928386
dtype: float64

<font size="2">$\hat{Y}=14,476.8-526.31age-0.019km-127.27seats-2,229.51fuel+1,103.13seller-206.8owner-124.82kmpl+0.288engine(cc)+82.93power(bhp)$</font>



In [100]:
#Export to CSV
df2.to_csv('Df2Updated')